In [3]:
%pip install pycryptodome
%pip install pycryptodome pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import subprocess
print(subprocess.getoutput("nvidia-smi"))

Fri Dec  5 07:40:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import gradio as gr
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
from Crypto.Util import number
import mimetypes, json, base64, io, os
from math import gcd, ceil
from PIL import Image

# ===================== Helper dasar =====================

def guess_mime(name: str):
    if not name:
        return "application/octet-stream"
    mt, _ = mimetypes.guess_type(name)
    return mt or "application/octet-stream"

MAGIC = b"CRPT"

def pack_blob(header: dict, payload: bytes) -> bytes:
    hjson = json.dumps(header).encode("utf-8")
    return MAGIC + len(hjson).to_bytes(4, "big") + hjson + payload

def unpack_blob(blob: bytes):
    if not blob.startswith(MAGIC):
        raise ValueError("Bukan file terenkripsi sistem ini (magic mismatch)")
    if len(blob) < 8:
        raise ValueError("Blob terlalu pendek")
    hlen = int.from_bytes(blob[4:8], "big")
    if 8 + hlen > len(blob):
        raise ValueError("Header length invalid")
    hjson = blob[8:8+hlen]
    header = json.loads(hjson.decode("utf-8"))
    payload = blob[8+hlen:]
    return header, payload

def aes_encrypt(plaintext: bytes):
    key = get_random_bytes(32)
    nonce = get_random_bytes(12)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    ct, tag = cipher.encrypt_and_digest(plaintext)
    return key, nonce, tag, ct

def aes_decrypt(key: bytes, nonce: bytes, tag: bytes, ciphertext: bytes):
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    return cipher.decrypt_and_verify(ciphertext, tag)

# ===================== RSA helper =====================

def generate_rsa_key(bits: int = 2048):
    key = RSA.generate(bits, e=65537)
    pub_str = f"{key.n},{key.e}"
    priv_str = f"{key.n},{key.d},{key.p},{key.q}"
    return pub_str, priv_str

def parse_rsa_pub(s: str):
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    if len(parts) != 2:
        raise ValueError("RSA public key harus format 'n,e'")
    return int(parts[0]), int(parts[1])

def parse_rsa_priv(s: str):
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    if len(parts) != 4:
        raise ValueError("RSA private key harus format 'n,d,p,q'")
    return int(parts[0]), int(parts[1]), int(parts[2]), int(parts[3])

def rsa_encrypt_bytes(rsa_pub_str: str, plaintext: bytes, orig_name: str = None, mime: str = None) -> bytes:
    n, e = parse_rsa_pub(rsa_pub_str)
    pub = RSA.construct((n, e))
    key, nonce, tag, ct = aes_encrypt(plaintext)
    oaep = PKCS1_OAEP.new(pub, hashAlgo=SHA256)
    wrapped = oaep.encrypt(key)
    header = {
        "scheme": "RSA",
        "algo": "RSA-OAEP+AES-GCM",
        "nonce": base64.b64encode(nonce).decode(),
        "tag": base64.b64encode(tag).decode(),
        "wrapped_key": base64.b64encode(wrapped).decode(),
        "orig_name": orig_name,
        "mime": mime,
    }
    return pack_blob(header, ct)

def rsa_decrypt_bytes(rsa_priv_str: str, blob: bytes):
    header, payload = unpack_blob(blob)
    if header.get("scheme") != "RSA":
        raise ValueError("Header scheme bukan RSA")
    n, d, p, q = parse_rsa_priv(rsa_priv_str)
    if p * q != n:
        raise ValueError("RSA private key tidak konsisten: p*q != n")
    priv = RSA.construct((n, 65537, d, p, q))
    oaep = PKCS1_OAEP.new(priv, hashAlgo=SHA256)
    nonce = base64.b64decode(header["nonce"])
    tag = base64.b64decode(header["tag"])
    wrapped = base64.b64decode(header["wrapped_key"])
    key = oaep.decrypt(wrapped)
    pt = aes_decrypt(key, nonce, tag, payload)
    return pt, header

# ===================== ElGamal helper =====================

def generate_safe_prime(bits=256):
    while True:
        q = number.getPrime(bits - 1)
        p = 2*q + 1
        if number.isPrime(p):
            return p

def find_generator(p):
    q = (p - 1)//2
    while True:
        g = number.getRandomRange(2, p-1)
        if pow(g, 2, p) != 1 and pow(g, q, p) != 1:
            return g

def elgamal_keygen(bits=512):
    p = generate_safe_prime(bits)
    g = find_generator(p)
    x = number.getRandomRange(2, p-2)
    y = pow(g, x, p)
    return p, g, y, x

def generate_elgamal_key(bits=512):
    p, g, y, x = elgamal_keygen(bits)
    pub_str = f"{p},{g},{y}"
    priv_str = f"{p},{g},{y},{x}"
    return pub_str, priv_str

def parse_elgamal_pub(s: str):
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    if len(parts) != 3:
        raise ValueError("ElGamal public key harus format 'p,g,y'")
    return int(parts[0]), int(parts[1]), int(parts[2])

def parse_elgamal_priv(s: str):
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    if len(parts) != 4:
        raise ValueError("ElGamal private key harus format 'p,g,y,x'")
    return int(parts[0]), int(parts[1]), int(parts[2]), int(parts[3])

def elgamal_encrypt_bytes(eg_pub_str: str, plaintext: bytes, orig_name: str = None, mime: str = None) -> bytes:
    p, g, y = parse_elgamal_pub(eg_pub_str)
    aes_key, nonce, tag, ct = aes_encrypt(plaintext)
    key_int = int.from_bytes(aes_key, 'big')
    if key_int >= p:
        key_int = key_int % (p - 1)
    k = number.getRandomRange(2, p-2)
    c1 = pow(g, k, p)
    s = pow(y, k, p)
    c2 = (key_int * s) % p
    header = {
        "scheme": "ElGamal",
        "algo": "ElGamal+AES-GCM",
        "nonce": base64.b64encode(nonce).decode(),
        "tag": base64.b64encode(tag).decode(),
        "c1": str(c1),
        "c2": str(c2),
        "orig_name": orig_name,
        "mime": mime,
    }
    return pack_blob(header, ct)

def elgamal_decrypt_bytes(eg_priv_str: str, blob: bytes):
    header, payload = unpack_blob(blob)
    if header.get("scheme") != "ElGamal":
        raise ValueError("Header scheme bukan ElGamal")
    p, g, y, x = parse_elgamal_priv(eg_priv_str)
    c1 = int(header["c1"])
    c2 = int(header["c2"])
    nonce = base64.b64decode(header["nonce"])
    tag = base64.b64decode(header["tag"])
    s = pow(c1, x, p)
    s_inv = pow(s, p-2, p)
    key_int = (c2 * s_inv) % p
    key_bytes = key_int.to_bytes((key_int.bit_length() + 7) // 8, 'big') if key_int > 0 else b'\x00'
    aes_key = key_bytes.rjust(32, b'\x00')
    pt = aes_decrypt(aes_key, nonce, tag, payload)
    return pt, header

# ===================== File I/O =====================

def read_uploaded_file(file_input):
    if file_input is None:
        return None, None, None
    path = str(file_input)
    filename = os.path.basename(path)
    with open(path, "rb") as f:
        data = f.read()
    mime = guess_mime(filename)
    return data, filename, mime

def blob_to_cipher_image(blob: bytes, out_name: str):
    total_len = len(blob)
    prefix = total_len.to_bytes(4, "big")
    raw = prefix + blob
    width = 256
    height = ceil(len(raw) / width)
    padded_len = width * height
    raw_padded = raw + b"\x00" * (padded_len - len(raw))
    img = Image.frombytes("L", (width, height), raw_padded)
    bio = io.BytesIO()
    img.save(bio, format="PNG")
    cipher_bytes = bio.getvalue()
    cipher_name = os.path.splitext(out_name)[0] + "_cipher.png"
    return cipher_name, cipher_bytes

def cipher_image_to_blob(file_input):
    data, filename, mime = read_uploaded_file(file_input)
    if data is None:
        raise ValueError("File cipher image tidak terbaca")
    bio = io.BytesIO(data)
    img = Image.open(bio).convert("L")
    raw = img.tobytes()
    if len(raw) < 4:
        raise ValueError("Cipher image terlalu kecil")
    total_len = int.from_bytes(raw[:4], "big")
    if 4 + total_len > len(raw):
        raise ValueError("Panjang blob di cipher image tidak valid")
    blob = raw[4:4+total_len]
    return blob, filename

def blob_to_cipher_text(blob: bytes, orig_name: str):
    b64 = base64.b64encode(blob).decode("ascii")
    cipher_name = os.path.splitext(orig_name)[0] + "_cipher.txt"
    return cipher_name, b64.encode("utf-8")

def cipher_text_to_blob(file_input):
    data, filename, mime = read_uploaded_file(file_input)
    if data is None:
        raise ValueError("File cipher txt tidak terbaca")
    try:
        b64 = data.decode("utf-8")
        blob = base64.b64decode(b64)
    except Exception as e:
        raise ValueError("Isi file cipher txt tidak valid base64") from e
    return blob, filename

# ===================== Status handlers =====================

def update_status_tab1(method, operation, file_kind):
    return f"Metode   : {method}\nOperasi  : {operation}\nJenis file: {file_kind}"

def update_status_tab2(start_method, file_kind):
    return f"Alur super mulai dari : {start_method}\nJenis file            : {file_kind}"

# ===================== Keygen handlers =====================

def rsa_gen_enc_click(rsa_pub_state, rsa_priv_state):
    if not rsa_pub_state or not rsa_priv_state:
        pub, priv = generate_rsa_key()
        status = "✅ RSA key pair baru di-generate. Public dipakai untuk ENCRYPT."
    else:
        pub, priv = rsa_pub_state, rsa_priv_state
        status = "ℹ️ RSA key pair yang sudah ada dipakai ulang."
    return pub, "", pub, priv, status

def rsa_gen_dec_click(rsa_pub_state, rsa_priv_state):
    if not rsa_pub_state or not rsa_priv_state:
        pub, priv = generate_rsa_key()
        status = "✅ RSA key pair baru di-generate. Private dipakai untuk DECRYPT."
    else:
        pub, priv = rsa_pub_state, rsa_priv_state
        status = "ℹ️ RSA key pair yang sudah ada dipakai ulang."
    return "", priv, pub, priv, status

def eg_gen_enc_click(eg_pub_state, eg_priv_state):
    if not eg_pub_state or not eg_priv_state:
        pub, priv = generate_elgamal_key()
        status = "✅ ElGamal key pair baru di-generate. Public dipakai untuk ENCRYPT."
    else:
        pub, priv = eg_pub_state, eg_priv_state
        status = "ℹ️ ElGamal key pair yang sudah ada dipakai ulang."
    return pub, "", pub, priv, status

def eg_gen_dec_click(eg_pub_state, eg_priv_state):
    if not eg_pub_state or not eg_priv_state:
        pub, priv = generate_elgamal_key()
        status = "✅ ElGamal key pair baru di-generate. Private dipakai untuk DECRYPT."
    else:
        pub, priv = eg_pub_state, eg_priv_state
        status = "ℹ️ ElGamal key pair yang sudah ada dipakai ulang."
    return "", priv, pub, priv, status

# ===================== Super Encryption handlers =====================

def super_gen_keys(super_rsa_pub_state, super_rsa_priv_state, super_eg_pub_state, super_eg_priv_state):
    if not super_rsa_pub_state or not super_rsa_priv_state:
        rsa_pub, rsa_priv = generate_rsa_key()
    else:
        rsa_pub, rsa_priv = super_rsa_pub_state, super_rsa_priv_state
    
    if not super_eg_pub_state or not super_eg_priv_state:
        eg_pub, eg_priv = generate_elgamal_key()
    else:
        eg_pub, eg_priv = super_eg_pub_state, super_eg_priv_state
    
    status = "✅ Keys RSA & ElGamal untuk Super Encryption sudah di-generate."
    return rsa_pub, rsa_priv, eg_pub, eg_priv, rsa_pub, rsa_priv, eg_pub, eg_priv, status

def process_super_encryption(start_method, file_kind, super_file, super_rsa_pub, super_rsa_priv, super_eg_pub, super_eg_priv):
    try:
        if super_file is None:
            return None, None, "❌ Harap upload file terlebih dahulu."
        
        if not super_rsa_pub.strip() or not super_rsa_priv.strip():
            return None, None, "❌ RSA keys kosong. Generate keys terlebih dahulu."
        if not super_eg_pub.strip() or not super_eg_priv.strip():
            return None, None, "❌ ElGamal keys kosong. Generate keys terlebih dahulu."
        
        data, filename, mime = read_uploaded_file(super_file)
        if data is None:
            return None, None, "❌ File tidak dapat dibaca."
        
        output_dir = "output_Super"
        os.makedirs(output_dir, exist_ok=True)
        
        if start_method == "RSA":
            blob1 = rsa_encrypt_bytes(super_rsa_pub, data, filename, mime)
            blob2 = elgamal_encrypt_bytes(super_eg_pub, blob1, filename + "_super", "application/octet-stream")
            method_order = "RSA→ElGamal"
            method_suffix = "RSA-ElGamal"
        else:
            blob1 = elgamal_encrypt_bytes(super_eg_pub, data, filename, mime)
            blob2 = rsa_encrypt_bytes(super_rsa_pub, blob1, filename + "_super", "application/octet-stream")
            method_order = "ElGamal→RSA"
            method_suffix = "ElGamal-RSA"
        
        base_name = os.path.splitext(filename)[0]
        if file_kind == "Text (.txt)":
            super_name = f"{base_name}_super_{method_suffix}_cipher.txt"
            _, cipher_bytes = blob_to_cipher_text(blob2, filename)
            super_bytes = cipher_bytes
        else:
            super_name = f"{base_name}_super_{method_suffix}_cipher.png"
            _, cipher_bytes = blob_to_cipher_image(blob2, filename)
            super_bytes = cipher_bytes
        
        super_path = os.path.join(output_dir, super_name)
        with open(super_path, "wb") as f:
            f.write(super_bytes)
        
        if start_method == "RSA":
            decrypted1, _ = elgamal_decrypt_bytes(super_eg_priv, blob2)
            decrypted2, header = rsa_decrypt_bytes(super_rsa_priv, decrypted1)
        else:
            decrypted1, _ = rsa_decrypt_bytes(super_rsa_priv, blob2)
            decrypted2, header = elgamal_decrypt_bytes(super_eg_priv, decrypted1)
        
        orig_name = header.get("orig_name") or "recovered.bin"
        recovered_path = os.path.join(output_dir, orig_name)
        with open(recovered_path, "wb") as f:
            f.write(decrypted2)
        
        if decrypted2 == data:
            integrity = "✅ VERIFIED: File recovered 100% identik dengan aslinya!"
        else:
            integrity = "⚠️ WARNING: File recovered berbeda dari aslinya!"
        
        status = f"✅ Super Encryption berhasil!\n"
        status += f"📊 Alur: {method_order}\n"
        status += f"📁 Super-encrypted: {super_name}\n"
        status += f"💾 Lokasi: {super_path}\n\n"
        status += f"✅ Super Decryption berhasil!\n"
        status += f"📄 Recovered: {orig_name}\n"
        status += f"💾 Lokasi: {recovered_path}\n"
        status += f"{integrity}"
        
        return super_path, recovered_path, status
    
    except Exception as e:
        return None, None, f"❌ Error (Super Encryption): {e}"

# ===================== Process Tab 1 =====================

def process_tab1(method, operation, file_kind, file_input, rsa_pub, rsa_priv, eg_pub, eg_priv):
    try:
        if file_input is None:
            return None, "❌ Harap upload file terlebih dahulu."
        data, filename, mime = read_uploaded_file(file_input)
        if data is None:
            return None, "❌ File tidak dapat dibaca."
        ext = (filename.rsplit(".", 1)[-1].lower() if "." in filename else "")
        if file_kind == "Text (.txt)" and ext != "txt":
            return None, f"❌ Jenis file dipilih 'Text', tapi ekstensi file adalah .{ext}."
        if file_kind == "Image (JPG/PNG/etc)" and not mime.startswith("image/"):
            return None, f"❌ Jenis file dipilih 'Image', tapi MIME file adalah {mime}."
        output_dir = "output_RSA" if method == "RSA" else "output_ElGamal"
        os.makedirs(output_dir, exist_ok=True)
        if operation == "Encrypt":
            if method == "RSA":
                if not rsa_pub.strip():
                    return None, "❌ RSA Public Key kosong. Generate atau isi dulu."
                blob = rsa_encrypt_bytes(rsa_pub, data, filename, mime)
            else:
                if not eg_pub.strip():
                    return None, "❌ ElGamal Public Key kosong. Generate atau isi dulu."
                blob = elgamal_encrypt_bytes(eg_pub, data, filename, mime)
            if file_kind == "Text (.txt)":
                out_name, out_bytes = blob_to_cipher_text(blob, filename)
            else:
                out_name, out_bytes = blob_to_cipher_image(blob, filename)
            output_path = os.path.join(output_dir, out_name)
            with open(output_path, "wb") as f:
                f.write(out_bytes)
            status = f"✅ Encrypt {method} berhasil!\n📁 Output: {out_name}\n💾 Disimpan di: {output_path}"
            return output_path, status
        else:
            if file_kind == "Text (.txt)":
                blob, cf_name = cipher_text_to_blob(file_input)
            else:
                blob, cf_name = cipher_image_to_blob(file_input)
            if method == "RSA":
                if not rsa_priv.strip():
                    return None, "❌ RSA Private Key kosong. Generate atau isi dulu."
                pt, header = rsa_decrypt_bytes(rsa_priv, blob)
            else:
                if not eg_priv.strip():
                    return None, "❌ ElGamal Private Key kosong. Generate atau isi dulu."
                pt, header = elgamal_decrypt_bytes(eg_priv, blob)
            orig_name = header.get("orig_name") or "decrypted.bin"
            output_path = os.path.join(output_dir, orig_name)
            with open(output_path, "wb") as f:
                f.write(pt)
            status = f"✅ Decrypt {method} berhasil!\n📄 File asli: {orig_name}\n💾 Disimpan di: {output_path}"
            return output_path, status
    except Exception as e:
        return None, f"❌ Error (process_tab1): {e}"

# ===================== GUI =====================

with gr.Blocks() as demo:
    gr.Markdown("## Hybrid RSA & ElGamal + Super Encryption")
    with gr.Tab("Tab 1: RSA / ElGamal"):
        with gr.Row():
            method = gr.Radio(["RSA", "ElGamal"], value="RSA", label="Metode")
            operation = gr.Radio(["Encrypt", "Decrypt"], value="Encrypt", label="Operasi")
            file_kind1 = gr.Radio(["Text (.txt)", "Image (JPG/PNG/etc)"], value="Text (.txt)", label="Jenis file input")
        file_input = gr.File(label="Input File (txt / image / dll)", file_count="single", type="filepath")
        gr.Markdown("### Parameter RSA")
        with gr.Row():
            rsa_pub = gr.Textbox(label="RSA Public Key (n,e) untuk ENCRYPT")
            rsa_priv = gr.Textbox(label="RSA Private Key (n,d,p,q) untuk DECRYPT")
        rsa_pub_state = gr.State("")
        rsa_priv_state = gr.State("")
        with gr.Row():
            rsa_gen_enc = gr.Button("Generate RSA Key for Encryption")
            rsa_gen_dec = gr.Button("Generate RSA Key for Decryption")
        gr.Markdown("### Parameter ElGamal")
        with gr.Row():
            eg_pub = gr.Textbox(label="ElGamal Public Key (p,g,y) untuk ENCRYPT")
            eg_priv = gr.Textbox(label="ElGamal Private Key (p,g,y,x) untuk DECRYPT")
        eg_pub_state = gr.State("")
        eg_priv_state = gr.State("")
        with gr.Row():
            eg_gen_enc = gr.Button("Generate ElGamal Key for Encryption")
            eg_gen_dec = gr.Button("Generate ElGamal Key for Decryption")
        run_btn = gr.Button("Process")
        output_file = gr.File(label="Output File (Encrypted / Decrypted)")
        status_tab1 = gr.Textbox(label="Status", lines=4)
        method.change(update_status_tab1, inputs=[method, operation, file_kind1], outputs=status_tab1)
        operation.change(update_status_tab1, inputs=[method, operation, file_kind1], outputs=status_tab1)
        file_kind1.change(update_status_tab1, inputs=[method, operation, file_kind1], outputs=status_tab1)
        run_btn.click(process_tab1, inputs=[method, operation, file_kind1, file_input, rsa_pub, rsa_priv, eg_pub, eg_priv], outputs=[output_file, status_tab1])
        rsa_gen_enc.click(rsa_gen_enc_click, inputs=[rsa_pub_state, rsa_priv_state], outputs=[rsa_pub, rsa_priv, rsa_pub_state, rsa_priv_state, status_tab1])
        rsa_gen_dec.click(rsa_gen_dec_click, inputs=[rsa_pub_state, rsa_priv_state], outputs=[rsa_pub, rsa_priv, rsa_pub_state, rsa_priv_state, status_tab1])
        eg_gen_enc.click(eg_gen_enc_click, inputs=[eg_pub_state, eg_priv_state], outputs=[eg_pub, eg_priv, eg_pub_state, eg_priv_state, status_tab1])
        eg_gen_dec.click(eg_gen_dec_click, inputs=[eg_pub_state, eg_priv_state], outputs=[eg_pub, eg_priv, eg_pub_state, eg_priv_state, status_tab1])
    with gr.Tab("Tab 2: Super Encryption"):
        with gr.Row():
            start_method = gr.Radio(["RSA", "ElGamal"], value="RSA", label="Mulai dari")
            file_kind2 = gr.Radio(["Text (.txt)", "Image (JPG/PNG/etc)"], value="Text (.txt)", label="Jenis file input")
        super_file = gr.File(label="File untuk Super Process", file_count="single", type="filepath")
        gr.Markdown("### Key RSA (Super)")
        super_rsa_pub = gr.Textbox(label="RSA Public (n,e)")
        super_rsa_priv = gr.Textbox(label="RSA Private (n,d,p,q)")
        super_rsa_pub_state = gr.State("")
        super_rsa_priv_state = gr.State("")
        gr.Markdown("### Key ElGamal (Super)")
        super_eg_pub = gr.Textbox(label="ElGamal Public (p,g,y)")
        super_eg_priv = gr.Textbox(label="ElGamal Private (p,g,y,x)")
        super_eg_pub_state = gr.State("")
        super_eg_priv_state = gr.State("")
        super_gen = gr.Button("Generate Keys for Super Encryption")
        super_run = gr.Button("Process Super")
        super_enc_file = gr.File(label="Super-encrypted (step2 result)")
        super_rec_file = gr.File(label="Recovered file (after 4 steps)")
        super_status = gr.Textbox(label="Status Super", lines=10)
        start_method.change(update_status_tab2, inputs=[start_method, file_kind2], outputs=super_status)
        file_kind2.change(update_status_tab2, inputs=[start_method, file_kind2], outputs=super_status)
        super_gen.click(
            super_gen_keys,
            inputs=[super_rsa_pub_state, super_rsa_priv_state, super_eg_pub_state, super_eg_priv_state],
            outputs=[super_rsa_pub, super_rsa_priv, super_eg_pub, super_eg_priv, super_rsa_pub_state, super_rsa_priv_state, super_eg_pub_state, super_eg_priv_state, super_status]
        )
        super_run.click(
            process_super_encryption,
            inputs=[start_method, file_kind2, super_file, super_rsa_pub, super_rsa_priv, super_eg_pub, super_eg_priv],
            outputs=[super_enc_file, super_rec_file, super_status]
        )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
